<a href="https://colab.research.google.com/github/jproctor-rebecca/lambdaprojects/blob/master/Spotify_recommender_NN_explore_RJProctor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports and enviroment setup

In [34]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.manifold import TSNE


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, RNN
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

## Load and clean data

In [38]:
# load data
# URL = 'https://github.com/Build-Week-Track-Team-7/explore/blob/main/data.csv.zip?raw=true'

# path_to_zip = tf.keras.utils.get_file('./data.zip', origin=URL, extract=True)
# PATH = os.path.join(os.path.dirname(path_to_zip), 'data')

data = pd.read_csv('./data.csv')
data.head()


,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.995,['Carl Woitschach'],0.708,158648,0.1950,0,6KbQ3uYMLKb5jDxLF7wYDD,0.563,10,0.1510,-12.428,1,Singende Bataillone 1. Teil,0,1928,0.0506,118.469,0.7790,1928
1,0.994,"['Robert Schumann', 'Vladimir Horowitz']",0.379,282133,0.0135,0,6KuQTIu1KoTTkLXKrwlLPV,0.901,8,0.0763,-28.454,1,"FantasiestÃ¼cke, Op. 111: PiÃ¹ tosto lento",0,1928,0.0462,83.972,0.0767,1928
2,0.604,['Seweryn GoszczyÅ„ski'],0.749,104300,0.2200,0,6L63VW0PibdM1HDSBoqnoM,0.000,5,0.1190,-19.924,0,Chapter 1.18 - Zamek kaniowski,0,1928,0.9290,107.177,0.8800,1928
3,0.995,['Francisco Canaro'],0.781,180760,0.1300,0,6M94FkXd15sOAOQYRnWPN8,0.887,1,0.1110,-14.734,0,Bebamos Juntos - Instrumental (Remasterizado),0,9/25/1928,0.0926,108.003,0.7200,1928
4,0.990,"['FrÃ©dÃ©ric Chopin', 'Vladimir Horowitz']",0.210,687733,0.2040,0,6N6tiFZ9vLTSOIxkj8qKrd,0.908,11,0.0980,-16.829,1,"Polonaise-Fantaisie in A-Flat Major, Op. 61",1,1928,0.0424,62.149,0.0693,1928


In [43]:
    
def clean_data():
  # drop columns adding bias and noise to dataset for our business problem
  # bias
  data_cleaned = data.drop('artists', axis=1)
    # duplicate/triplicate information may disproportantly weight results
  data_cleaned = data_cleaned.drop('release_date', axis=1)
    # duplicate/triplicate information may disproportantly weight results

  # noise
  data_cleaned = data_cleaned.drop('id', axis=1)
    # has no meaningful contribution to results

  return data_cleaned


clean_data()
print(data_cleaned.shape)
data_cleaned.head()

(169909, 18)


,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,speechiness,tempo,valence,year
0,0.995,['Carl Woitschach'],0.708,158648,0.1950,0,6KbQ3uYMLKb5jDxLF7wYDD,0.563,10,0.1510,-12.428,1,Singende Bataillone 1. Teil,0,0.0506,118.469,0.7790,1928
1,0.994,"['Robert Schumann', 'Vladimir Horowitz']",0.379,282133,0.0135,0,6KuQTIu1KoTTkLXKrwlLPV,0.901,8,0.0763,-28.454,1,"FantasiestÃ¼cke, Op. 111: PiÃ¹ tosto lento",0,0.0462,83.972,0.0767,1928
2,0.604,['Seweryn GoszczyÅ„ski'],0.749,104300,0.2200,0,6L63VW0PibdM1HDSBoqnoM,0.000,5,0.1190,-19.924,0,Chapter 1.18 - Zamek kaniowski,0,0.9290,107.177,0.8800,1928
3,0.995,['Francisco Canaro'],0.781,180760,0.1300,0,6M94FkXd15sOAOQYRnWPN8,0.887,1,0.1110,-14.734,0,Bebamos Juntos - Instrumental (Remasterizado),0,0.0926,108.003,0.7200,1928
4,0.990,"['FrÃ©dÃ©ric Chopin', 'Vladimir Horowitz']",0.210,687733,0.2040,0,6N6tiFZ9vLTSOIxkj8qKrd,0.908,11,0.0980,-16.829,1,"Polonaise-Fantaisie in A-Flat Major, Op. 61",1,0.0424,62.149,0.0693,1928


In [41]:
# create feature matrix
X = data_cleaned.drop('name', axis=1)
y = data_cleaned['name']

# split data - train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    random_state=42
                                                    )

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(135927, 17)
(135927,)
(33982, 17)
(33982,)


## Additional preproccessing of data
Pipeline 1 - (instantiate, fit_transform)

*   OneHotEncoder
*   OrdinalEncoder
*   StandardScaler

Pipeline 2, 3, 4 - (instantiate, fit, predict)

*   KNN
*   PCA
*   t_SNE

#### Pipeline 1

In [ ]:
# normalize standardize data using pipeline (instantiate, fit, transform)
# onehotender, ordinal encoder, standard scaler in order



# Explore each to reduce dimensionality of data: KNN, PCA and t-SNE (instantiate, fit, predict) (tune after MVPs met)




#### Pipeline 2

#### Pipeline 3

#### Pipeline 4

#### Transform data to numpy array to feed into NN

In [ ]:
# transform data to np array


## Neural Network model 
Pipeline 1 - (instantiate, compile, fit)

*   RNN
*   LSTM
*   RNN (deep and wide)

## Evaluate model

## Vizualize results

## Conclusions

## Export to application

In [ ]:
# convert to json
